### Import Required Libraries and Set Up Environment Variables

In [48]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import time

In [49]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [50]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [51]:
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    page_query_url = query_url + f"&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url).json()
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews_list.append(review)
        
        # Print the page that was just retrieved
        print(f"Retrieved page {page}")
    
    # Print the page number that had no results then break from the loop
    except: 
        print(f"No results for page {page}")
        break

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)


Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19


In [67]:
# Define the variable "reviews" and assign it the necessary data
reviews = response["response"]["docs"]

# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2017/05/11/movies/hounds-of-love-review.html",
        "snippet": "This feature debut of the Australian filmmaker Ben Young resembles a real-life couple\u2019s killing spree in the 1980s.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: \u2018Hounds of Love\u2019 Is Tense and Deadly Down Under",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Hounds of Love",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Hounds of Love (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
     

In [53]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [54]:
# Check if 'headline.main' column exists in 'reviews_df'
if 'headline.main' in reviews_df.columns:
    # Extract the title from the "headline.main" column and save it to a new column "title"
    # Title is between unicode characters \u2018 and \u2019. 
    # End string should include " Review" to avoid cutting title early
    reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
    
    print(reviews_df[["headline.main", "title"]].head())
else:
    # Handle the case when 'headline.main' column does not exist
    print("Error: 'headline.main' column does not exist in 'reviews_df'")


                                       headline.main  \
0        ‘The Attachment Diaries’ Review: Love, Sick   
1  Review: ‘What’s Love Got to Do With It?’ Proba...   
2  ‘You Can Live Forever’ Review: Do You Love Me ...   
3  ‘A Tourist’s Guide to Love’ Review: A Wearying...   
4     ‘Other People’s Children’ Review: True Romance   

                                           title  
0                         The Attachment Diaries  
1  What’s Love Got to Do With It?’ Probably a Lo  
2                           You Can Live Forever  
3                      A Tourist’s Guide to Love  
4                        Other People’s Children  


In [55]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
# Fix the "keywords" column by converting cells from a list to a string

reviews_df["keywords"].head()


0    subject: Movies;creative_works: The Attachment...
1    subject: Movies;persons: Kapur, Shekhar;person...
2    subject: Movies;creative_works: You Can Live F...
3    subject: Movies;creative_works: A Tourist's Gu...
4    subject: Movies;persons: Zlotowski, Rebecca;cr...
Name: keywords, dtype: object

In [56]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()
for title in titles:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie
Love Is Love Is Love
Love Hard
Bergman Island
Hard Luck Love Song
South of Heaven
Wife of a Spy
Happier Than Ever
Together
Annette
Resort t

### Access The Movie Database API

In [57]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [58]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        # Sleep for 12 seconds before making a request
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    movie = requests.get(url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie_response = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genre_names = [genre["name"] for genre in movie_response["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language["english_name"] for language in movie_response["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_response["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_response["title"],
            "genre_names": genre_names,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })
        
        # Print out the title that was found
        print(f"Movie found: {title}")
    except:
        # Print out a statement if a movie is not found
        print(f"Movie not found: {title}")
        production_countries = []

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_response["title"],
            "genre_names": genre_names,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })
        
        # Print out the title that was found
        print(f"Movie found: {title}")


Movie found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Movie found: What’s Love Got to Do With It?’ Probably a Lo
Movie found: You Can Live Forever
Movie found: A Tourist’s Guide to Love
Movie found: Other People’s Children
Movie found: One True Loves
Movie found: The Lost Weekend: A Love Story
Movie found: A Thousand and One
Movie found: Your Place or Mine
Movie found: Love in the Time of Fentanyl
Movie found: Pamela, a Love Story
Movie found: In From the Side
Movie found: After Love
Movie found: Alcarràs
Movie found: Nelly & Nadine
Movie found: Lady Chatterley’s Lover
Movie found: The Sound of Christmas
Movie found: The Inspection
Movie found: Bones and All
Movie found: My Policeman
Movie found: About Fate
Movie found: Waiting for Bojangles
Movie found: I Love My Dad
Movie found: A Love Song
Movie found: Alone Together
Movie found: Art of Love
Movie found: The Wheel
Movie found: Thor: Love and Thunder
Movie found: Both Sides of the Blade
Mo

In [59]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "genre_names": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "The Attachment Diaries",
        "genre_names": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": []
    },
    {
        "title": "You Can Live Forever",
        "genre_names": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "French",
            "English"
        ],
        "production_countries": [
            "Canada"
        ]
    },
    {
        "title": "A Tourist's Guide to Love",
        "genre_names": [
            "Romance",
            "Comedy",
  

In [60]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)

# Display the head of the DataFrame
tmdb_movies_df.head()

,title,genre_names,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[]
2,You Can Live Forever,"[Drama, Romance]","[French, English]",[Canada]
3,A Tourist's Guide to Love,"[Romance, Comedy, Adventure]",[English],[United States of America]
4,Other People's Children,[Drama],[French],[France]


### Merge and Clean the Data for Export

In [61]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title")

In [62]:
# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
columns_to_fix = ["genre_names", "spoken_languages", "production_countries"]

for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
merged_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"Drama, Romance","French, English",Canada
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"Romance, Drama, Comedy",English,"Germany, United States of America"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,Documentary,English,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,Comedy,French,
163,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,Comedy,French,
164,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,Comedy,French,
165,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible 

In [63]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

In [64]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,"Drama, Romance","French, English",Canada
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None,One True Loves,"Romance, Drama, Comedy",English,"Germany, United States of America"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,Documentary,English,United States of America


In [65]:
# Export data to CSV without the index
import os
# Get the current working directory
current_dir = os.getcwd()

# Define the folder name
folder_name = "output_data"

# Create the output_data directory if it doesn't exist in the current working directory
output_dir = os.path.join(current_dir, folder_name)
os.makedirs(output_dir, exist_ok=True)

# Save the merged DataFrame as a CSV file in the output_data directory
output_file = os.path.join(output_dir, "merged_data.csv")

# Save the DataFrame to a CSV file
try:
    merged_df.to_csv(output_file, index=False)
    print("DataFrame successfully saved as CSV.")
except Exception as e:
    print("An error occurred:", str(e))


DataFrame successfully saved as CSV.
